In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
import copy
import inspect
import warnings
import cProfile

from astropy.table import Table, Column, vstack

import numpy as np 

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection

import palettable
color_fancy = palettable.wesanderson.Mendl_4.mpl_colors
color_bins = palettable.cartocolors.qualitative.Bold_4_r.mpl_colors

### Learn how to use `astroplan` and `astropy` for observing planning

* [`astroplan` can be found here](https://astroplan.readthedocs.io/en/latest/)

In [75]:
from astroplan import download_IERS_A
download_IERS_A()

In [3]:
import operator

from collections.abc import Sequence

from astroplan import Observer
from astroplan import FixedTarget
from astroplan.plots import plot_airmass, dark_style_sheet

from pytz import timezone

import astropy.units as u
from astropy.time import Time, TimezoneInfo
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.coordinates import get_sun, get_moon

from matplotlib import dates

In [4]:
# Site
ctio_timezone = timezone('Etc/GMT-4')
ctio = Observer.at_site('CTIO', timezone=ctio_timezone)

# Target
g09 = FixedTarget(name='G09', coord=SkyCoord(133.5, 0.95, unit='deg', frame='icrs'))
g12 = FixedTarget(name='G12', coord=SkyCoord(179.5, 0.00, unit='deg', frame='icrs'))
g15 = FixedTarget(name='G15', coord=SkyCoord(216.5, 0.00, unit='deg', frame='icrs'))
xmm1 = FixedTarget(name='XMM1', coord=SkyCoord(335.0, 2.50, unit='deg', frame='icrs'))
xmm2 = FixedTarget(name='XMM2', coord=SkyCoord(35.0, 2.50, unit='deg', frame='icrs'))

# Planned observing time (UTC)
obs_time = Time('2021-12-01 00:00:00', scale='utc')

obs_1 = [Time('2021-{0}-01 00:00:00'.format(month), scale='utc') for month in np.arange(12) + 1]
obs_2 = [Time('2021-{0}-15 00:00:00'.format(month), scale='utc') for month in np.arange(12) + 1]
obs_dates = obs_1 + obs_2

merian = [g09, g12, g15, xmm1, xmm2]

In [6]:
for obs_time in obs_dates:
    fig = airmass_plot(
        merian, ctio, obs_time, ax=None, min_airmass=0.95, max_airmass=3.2,
        min_region=1.5, max_region=2.0, show_legend=True, show_sun=False,
        show_moon=True, linewidth=3.0, color_cycle=None, gmt=-4)
    
    fig.savefig('fig/merian_plan_{0}_{1}_{2}.png'.format(
        obs_time.datetime.date().year, obs_time.datetime.date().month,
        obs_time.datetime.date().day), dpi=100)
    
    plt.close(fig)

In [5]:
def airmass_plot(targets, observer, time, ax=None, min_airmass=1.0, max_airmass=3.2,
                 min_region=None, max_region=None, show_legend=True, show_sun=False,
                 show_moon=True, linewidth=3.0, color_cycle=None, gmt=0):
    """Plot the airmass of targets given observing site and date."""
    if ax is None:
        fig = plt.figure(figsize=(15.5, 6))
        fig.subplots_adjust(left=0.06, right=0.95, bottom=0.135, top=0.865, wspace=0.0, hspace=0.0)
        ax = fig.add_subplot(111)
    
    if color_cycle is None:
        ax.set_prop_cycle('color', palettable.cartocolors.qualitative.Safe_10.mpl_colors)

    # Populate time window if needed.
    time = Time(time)
    if time.isscalar:
        time = time + np.linspace(-12.5, 12.5, 200) * u.hour - gmt * u.hour
    elif len(time) == 1:
        warnings.warn('You used a Time array of length 1.  You probably meant '
                      'to use a scalar. (Or maybe a list with length > 1?).')

    if not isinstance(targets, Sequence):
        targets = [targets]

    # Shade background during night time
    start = time[0].datetime

    # Calculate and order twilights and set plotting alpha for each
    twilights = [
        (observer.sun_set_time(Time(start), which='next').datetime, 0.1),
        (observer.twilight_evening_civil(Time(start), which='next').datetime, 0.2),
        (observer.twilight_evening_nautical(Time(start), which='next').datetime, 0.3),
        (observer.twilight_evening_astronomical(Time(start), which='next').datetime, 0.4),
        (observer.twilight_morning_astronomical(Time(start), which='next').datetime, 0.5),
        (observer.twilight_morning_nautical(Time(start), which='next').datetime, 0.4),
        (observer.twilight_morning_civil(Time(start), which='next').datetime, 0.3),
        (observer.sun_rise_time(Time(start), which='next').datetime, 0.2),
    ]

    twilights.sort(key=operator.itemgetter(0))
    for i, twi in enumerate(twilights[1:], 1):
        ax.axvspan(twilights[i - 1][0], twilights[i][0],
                   ymin=0, ymax=1, color='grey', alpha=twi[1])

    # Plot the Sun
    if show_sun:
        sun_airmass = observer.sun_altaz(time).secz
        sun_airmass = np.ma.array(sun_airmass, mask=sun_airmass < 1)
        ax.plot_date(time.plot_date, sun_airmass, label=r'$\rm Sun$', fmt='-', 
                     linestyle='-', linewidth=10.0, alpha=0.15, color='maroon')

    # Plot the Moon
    if show_moon:
        moon_airmass = observer.moon_altaz(time).secz
        moon_airmass = np.ma.array(moon_airmass, mask=moon_airmass < 1)
        moon_illum = np.mean(observer.moon_illumination(time))
        ax.plot_date(time.plot_date, moon_airmass, label=r'$\rm Moon$', fmt='--', 
                     linestyle='--', linewidth=5.0 * moon_illum, alpha=0.9, color='grey')

    for target in targets:
        # Calculate airmass
        airmass = observer.altaz(time, target).secz

        # Mask out nonsense airmasses
        masked_airmass = np.ma.array(airmass, mask=airmass < 1)

        # Some checks & info for labels.
        try:
            target_name = target.name
        except AttributeError:
            target_name = ''

        ax.plot_date(time.plot_date, masked_airmass, label='__no_label__',
                     fmt='-', linestyle='-', linewidth=3.4, alpha=0.9, color='k')
        ax.plot_date(time.plot_date, masked_airmass, label=r'\rm {:s}'.format(target_name),
                     fmt='-', linestyle='-', linewidth=3.0, alpha=0.9)

    if show_legend:
        ax.legend(loc='upper right', fontsize=20, fancybox=True, framealpha=0.8)

    date_formatter = dates.DateFormatter(r'$%H\colon%M$')
    ax.xaxis.set_major_formatter(date_formatter)

    # Format the time axis
    time_0 = (Time(twilights[0][0]) - 4.5 * u.hour).plot_date
    time_1 = (Time(twilights[-1][0]) + 4.5 * u.hour).plot_date
    ax.set_xlim([time_0, time_1])

    # Invert y-axis and set limits.
    y_lim = ax.get_ylim()
    if y_lim[1] > y_lim[0]:
        ax.invert_yaxis()
    ax.set_ylim([max_airmass, min_airmass])

    # Draw lo/hi limit regions, if present
    ymax, ymin = ax.get_ylim()       # should be (hi_limit, lo_limit)
    
    # Airmass limit
    if max_region is not None:
        ax.axhspan(ymax, max_region, facecolor='slategrey', alpha=0.10, zorder=0,
                   rasterized=True)
    if min_region is not None:
        ax.axhspan(min_region, ymin, facecolor='palegreen', alpha=0.10, zorder=0,
                   rasterized=True)

    # Altitude axis
    altitude_ticks = np.array([70, 50, 40, 30, 20, 10])
    airmass_ticks = 1. / np.cos(np.radians(90 - altitude_ticks))
    ax2 = ax.twinx()
    ax2.grid(False)
    ax2.invert_yaxis()
    ax2.set_yticks(airmass_ticks)
    ax2.set_yticklabels([r'${:d}$'.format(alt) for alt in altitude_ticks])
    ax2.set_ylim(ax.get_ylim())
    ax2.set_ylabel(r'$\rm Altitude\ [degrees]$', fontsize=25, labelpad=24, 
                   rotation=270.)

    # Local time labels
    ax3 = ax.twiny()
    ax3.set_xticks(ax.get_xticks() + gmt / 24.0)
    ax3.xaxis.set_major_formatter(date_formatter)
    ax3.set_xlim([time_0  + gmt / 24.0, time_1 + gmt / 24.0])

    if gmt < 0:
        _ = ax3.set_xlabel(
            r"$\rm Local\ Time\ [GMT{{{{\mbox{{{{-}}}}}}}}{0}]$".format(-gmt), 
            fontsize=25, labelpad=10)
    else:
        _ = ax3.set_xlabel(
            r"$\rm Local\ Time\ [GMT{{{{\mbox{{{{+}}}}}}}}{0}]$".format(gmt), 
            fontsize=25, labelpad=10)

    date_begin = min(time).datetime.date()
    _ = ax.set_xlabel(
        r"$\rm Time\ from\ {0}{{{{\mbox{{{{-}}}}}}}}{1}{{{{\mbox{{{{-}}}}}}}}{2}\ [UTC]$".format(
            date_begin.year, date_begin.month, date_begin.day), fontsize=25)
    _ = ax.set_ylabel(r"$\rm Airmass$", fontsize=25)
    
    return fig